In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
!pip install scikit-surprise
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import GridSearchCV
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/Colab Notebooks/Masters Python Workbooks/Redone Python Workbooks
books = pd.read_csv("Books.csv")
ratings = pd.read_csv("Ratings.csv")
users = pd.read_csv("Users.csv")
books["Book-Author"].fillna("Edinburgh Financial Publishing",inplace=True)
# We can replace the publisher name in these cases as well

# The publisher of 'Tyrant Moon' as per Amazon and FantasticFiction.com is 'Mundania Press LLC'
books['Publisher'].fillna("Mundania Press LLC", inplace=True)

# The publisher of 'Finders Keepers' as per GoodReads.com and Amazon is 'Bantam'
books['Publisher'].fillna("Bantam", inplace=True)

# Replacing the 0 aged users with nans so that they can be replaced in the next step
users.Age.replace(0,np.nan,inplace=True)

# Linear Interpolation of age in the dataset
users['Age'].interpolate(method = 'linear', limit_direction = 'forward',inplace=True)

# Dropping the rows with age > 100
users = users[users.Age <= 100]
book_rating = pd.merge(books,ratings,on='ISBN',how='outer')
book_rating = pd.merge(book_rating, users, on='User-ID', how='outer')
book_rating.drop(['Image-URL-S','Image-URL-M','Image-URL-L'],axis=1,inplace=True)
temp = book_rating[['ISBN','User-ID','Book-Rating']]
counts_user = temp.groupby('User-ID').agg(['count'])
counts_user.columns = ['ISBN_ct','rating_users_ct']
counts_book = temp.groupby('ISBN').agg(['count'])
counts_book.columns = ['ISBN_ct','rating_books_ct']
counts_user.drop(['ISBN_ct'], inplace=True, axis=1)
counts_book.drop(['ISBN_ct'], inplace=True, axis=1)
book_rating = pd.merge(book_rating, counts_book, on='ISBN',how='left')
book_rating = pd.merge(book_rating, counts_user, on='User-ID',how='left')

book_rating['City'] = book_rating['Location'].str.split(",",expand=True)[0]
book_rating['State'] = book_rating['Location'].str.split(",",expand=True)[1]
book_rating['Country'] = book_rating['Location'].str.split(",",expand=True)[2]
book_rating.drop('Location',axis=1,inplace=True)
df = book_rating.copy()

index_year = df[(df['Year-Of-Publication'] == 'DK Publishing Inc') | (df['Year-Of-Publication'] == 'Gallimard')].index
df.drop(index_year,inplace=True)
df = df.dropna(subset=['ISBN'])
df = df.dropna(subset=['Book-Title'])

# Dropping null values in User-ID as there could be no results if the user ID is not present
df = df.dropna(subset=['User-ID'])

# Deleting missing rows present in location as those are only 0.3% of the dataset
df.dropna(inplace=True)
# df.info()
# pd.isnull(df).sum()
df['Year-Of-Publication'] = df['Year-Of-Publication'].astype(int)

# One of the city names is '&#304;stanbul' which will be replaced with 'Istanbul'
df['City'].replace('&#304;stanbul','Istanbul',inplace=True)
df[df['Book-Title'] == 'Wild Animus']['ISBN'].value_counts()
most_popular_books = df[(df['rating_books_ct'] >= 568) & (df['rating_books_ct'] <= 2502.0)]
most_popular_books = most_popular_books[['ISBN','Book-Title','Book-Author','Publisher','rating_books_ct']]
most_popular_books = most_popular_books.drop_duplicates()
most_popular_books.shape

# books = list(most_popular_books['Book-Title'].values)
# temp = df.copy()
# df = temp[~temp['Book-Title'].isin(books)]
label_encoder = LabelEncoder()
# Creating a copy that will contain all the label encoders instead of the actual categorical variables
df1 = df.copy()
df1['City'] = label_encoder.fit_transform(df1['City'])
columns = ['Book-Author','Book-Title','Year-Of-Publication','Publisher','State','Country']
df1['State'] = label_encoder.fit_transform(df1['State'])
df1['Book-Author'] = label_encoder.fit_transform(df1['Book-Author'])
df1['Book-Title'] = label_encoder.fit_transform(df1['Book-Title'])
df1['Year-Of-Publication'] = df1['Year-Of-Publication'].astype(str)
df1['Year-Of-Publication'] = label_encoder.fit_transform(df1['Year-Of-Publication'])
df1['Publisher'] = label_encoder.fit_transform(df1['Publisher'])
df1['Country'] = label_encoder.fit_transform(df1['Country'])
# Creating another dataset that has only non categorical values that need to be scaled
numerical_df = df1[['Book-Title','Book-Author','Year-Of-Publication','Publisher','Book-Rating',
                    'Age','rating_books_ct','rating_users_ct','City','State','Country']] 
scaler = StandardScaler()
df_scaled = scaler.fit_transform(numerical_df)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = ['Book-Title','Book-Author','Year-Of-Publication','Publisher','Book-Rating',
                    'Age','rating_books_ct','rating_users_ct','City','State','Country']

# df is the dataset to use from Part A

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

/content/drive/MyDrive/Colab Notebooks/Masters Python Workbooks/Redone Python Workbooks


In [ ]:
# Hopkins Statistic for KMeans

# from sklearn.neighbors import NearestNeighbors
# from random import sample
# from numpy.random import uniform
# import numpy as np
# from math import isnan
 
# def hopkins(X):
#     d = X.shape[1]
#     #d = len(vars) # columns
#     n = len(X) # rows
#     m = int(0.1 * n) 
#     nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
#     rand_X = sample(range(0, n, 1), m)
 
#     ujd = []
#     wjd = []
#     for j in range(0, m):
#         u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
#         ujd.append(u_dist[0][1])
#         w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
#         wjd.append(w_dist[0][1])
 
#     H = sum(ujd) / (sum(ujd) + sum(wjd))
#     if isnan(H):
#         print(ujd, wjd)
#         H = 0
 
#     return H
# hopkins(df_scaled)

0.9192442172243398

In [ ]:
# model with k = 6
kmeans = KMeans(n_clusters=6, max_iter=50, random_state=0)
kmeans.fit(df_scaled)
df_6 = df.copy()
# assign the label
df_6['cluster_id'] = kmeans.labels_


# 6 clusters
df_6_0 = df_6[df_6['cluster_id'] == 0]
df_6_1 = df_6[df_6['cluster_id'] == 1]
df_6_2 = df_6[df_6['cluster_id'] == 2]
df_6_3 = df_6[df_6['cluster_id'] == 3]
df_6_4 = df_6[df_6['cluster_id'] == 4]
df_6_5 = df_6[df_6['cluster_id'] == 5]

print("Ready for collaborative filtering!")

Ready for collaborative filtering!


In [ ]:
df_6_0['Book-Rating'] = df_6_0['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_6_1['Book-Rating'] = df_6_1['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_6_2['Book-Rating'] = df_6_2['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_6_3['Book-Rating'] = df_6_3['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_6_4['Book-Rating'] = df_6_4['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_6_5['Book-Rating'] = df_6_5['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_0['Book-Rating'] = df_8_0['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_1['Book-Rating'] = df_8_1['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_2['Book-Rating'] = df_8_2['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_3['Book-Rating'] = df_8_3['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_4['Book-Rating'] = df_8_4['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_5['Book-Rating'] = df_8_5['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_6['Book-Rating'] = df_8_6['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
df_8_7['Book-Rating'] = df_8_7['Book-Rating'].apply(lambda x: 0 if x>=1 else 1)
for i in [df_6_0, df_6_1, df_6_2, df_6_3, df_6_4, df_6_5]:
  i.drop_duplicates()
  print(i.shape)

(268468, 14)
(267184, 14)
(3789, 14)
(262048, 14)
(34835, 14)
(191835, 14)


# Collaborative filtering
## PART B: User Similarity

### Clusters: 6

In [ ]:
print("df_6_0: ",df_6_0.shape)
print("df_6_1: ",df_6_1.shape)
print("df_6_2: ",df_6_2.shape)
print("df_6_3: ",df_6_3.shape)
print("df_6_4: ",df_6_4.shape)
print("df_6_5: ",df_6_5.shape)

df_6_0:  (268468, 14)
df_6_1:  (267184, 14)
df_6_2:  (3789, 14)
df_6_3:  (262048, 14)
df_6_4:  (34835, 14)
df_6_5:  (191835, 14)


###### Breaking df_6_2, df_6_3 and df_6_5

In [ ]:
# df_6_0
df_6_0_1 = np.array_split(df_6_0,10)[0]
df_6_0_2 = np.array_split(df_6_0,10)[1]
df_6_0_3 = np.array_split(df_6_0,10)[2]
df_6_0_4 = np.array_split(df_6_0,10)[3]
df_6_0_5 = np.array_split(df_6_0,10)[4]
df_6_0_6 = np.array_split(df_6_0,10)[5]
df_6_0_7 = np.array_split(df_6_0,10)[6]
df_6_0_8 = np.array_split(df_6_0,10)[7]
df_6_0_9 = np.array_split(df_6_0,10)[8]
df_6_0_10 = np.array_split(df_6_0,10)[9]

# df_6_1
df_6_1_1 = np.array_split(df_6_1,10)[0]
df_6_1_2 = np.array_split(df_6_1,10)[1]
df_6_1_3 = np.array_split(df_6_1,10)[2]
df_6_1_4 = np.array_split(df_6_1,10)[3]
df_6_1_5 = np.array_split(df_6_1,10)[4]
df_6_1_6 = np.array_split(df_6_1,10)[5]
df_6_1_7 = np.array_split(df_6_1,10)[6]
df_6_1_8 = np.array_split(df_6_1,10)[7]
df_6_1_9 = np.array_split(df_6_1,10)[8]
df_6_1_10 = np.array_split(df_6_1,10)[9]


# df_6_3
df_6_3_1 = np.array_split(df_6_3,15)[0]
df_6_3_2 = np.array_split(df_6_3,15)[1]
df_6_3_3 = np.array_split(df_6_3,15)[2]
df_6_3_4 = np.array_split(df_6_3,15)[3]
df_6_3_5 = np.array_split(df_6_3,15)[4]
df_6_3_6 = np.array_split(df_6_3,15)[5]
df_6_3_7 = np.array_split(df_6_3,15)[6]
df_6_3_8 = np.array_split(df_6_3,15)[7]
df_6_3_9 = np.array_split(df_6_3,15)[8]
df_6_3_10 = np.array_split(df_6_3,15)[9]
df_6_3_11 = np.array_split(df_6_3,15)[10]
df_6_3_12 = np.array_split(df_6_3,15)[11]
df_6_3_13 = np.array_split(df_6_3,15)[12]
df_6_3_14 = np.array_split(df_6_3,15)[13]
df_6_3_15 = np.array_split(df_6_3,15)[14]

# df_6_5
df_6_5_1 = np.array_split(df_6_5,20)[0]
df_6_5_2 = np.array_split(df_6_5,20)[1]
df_6_5_3 = np.array_split(df_6_5,20)[2]
df_6_5_4 = np.array_split(df_6_5,20)[3]
df_6_5_5 = np.array_split(df_6_5,20)[4]
df_6_5_6 = np.array_split(df_6_5,20)[5]
df_6_5_7 = np.array_split(df_6_5,20)[6]
df_6_5_8 = np.array_split(df_6_5,20)[7]
df_6_5_9 = np.array_split(df_6_5,20)[8]
df_6_5_10 = np.array_split(df_6_5,20)[9]
df_6_5_11 = np.array_split(df_6_5,20)[10]
df_6_5_12 = np.array_split(df_6_5,20)[11]
df_6_5_13 = np.array_split(df_6_5,20)[12]
df_6_5_14 = np.array_split(df_6_5,20)[13]
df_6_5_15 = np.array_split(df_6_5,20)[14]
df_6_5_16 = np.array_split(df_6_5,20)[15]
df_6_5_17 = np.array_split(df_6_5,20)[16]
df_6_5_18 = np.array_split(df_6_5,20)[17]
df_6_5_19 = np.array_split(df_6_5,20)[18]
df_6_5_20 = np.array_split(df_6_5,20)[19]

In [ ]:
df_6_features_2 = df_6_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_4 = df_6_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)

### Function to do user similarity automatically

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from numpy import *

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def user_similarity(df_features, original_df):
  mean = np.nanmean(df_features, axis=1)
  df_subtracted = (df_features.T-mean).T
# User Similarity Matrix
  user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
  user_correlation[np.isnan(user_correlation)] = 0
  user_correlation[user_correlation<0]=0
  user_predicted_ratings = np.dot(user_correlation, df_features.fillna(0))
  user_final_rating = np.multiply(user_predicted_ratings,df_features)

  X  = user_final_rating.copy() 
  X = X[X>0]
  scaler = MinMaxScaler(feature_range=(1, 5))
  scaler.fit(X)
  y = (scaler.transform(X))
  df_ = original_df.pivot_table(index='User-ID',columns='ISBN',values='Book-Rating')
  # Finding total non-NaN value
  total_non_nan = np.count_nonzero(~np.isnan(y))
  rmse = (sum(sum((df_ - y )**2))/total_non_nan)**0.5
  print("The RMSE of {}: {}".format(get_df_name(original_df), rmse))

In [ ]:
df_6_features_0_1 = df_6_0_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_2 = df_6_0_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_3 = df_6_0_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_4 = df_6_0_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_5 = df_6_0_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_6 = df_6_0_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_7 = df_6_0_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_8 = df_6_0_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_9 = df_6_0_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_0_10 = df_6_0_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
user_similarity(df_6_features_0_1, df_6_0_1)
user_similarity(df_6_features_0_2, df_6_0_2)
user_similarity(df_6_features_0_3, df_6_0_3)
user_similarity(df_6_features_0_4, df_6_0_4)
user_similarity(df_6_features_0_5, df_6_0_5)
user_similarity(df_6_features_0_6, df_6_0_6)
user_similarity(df_6_features_0_7, df_6_0_7)
user_similarity(df_6_features_0_8, df_6_0_8)
user_similarity(df_6_features_0_9, df_6_0_9)

df_6_features_1_1 = df_6_1_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_2 = df_6_1_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_3 = df_6_1_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_4 = df_6_1_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_5 = df_6_1_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_6 = df_6_1_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_7 = df_6_1_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_8 = df_6_1_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_9 = df_6_1_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_1_10 = df_6_1_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
user_similarity(df_6_features_1_1, df_6_1_1)
user_similarity(df_6_features_1_2, df_6_1_2)
user_similarity(df_6_features_1_3, df_6_1_3)
user_similarity(df_6_features_1_4, df_6_1_4)
user_similarity(df_6_features_1_5, df_6_1_5)
user_similarity(df_6_features_1_6, df_6_1_6)
user_similarity(df_6_features_1_7, df_6_1_7)
user_similarity(df_6_features_1_8, df_6_1_8)
user_similarity(df_6_features_1_9, df_6_1_9)

The RMSE of df_6_0_1: 0.5562409222280784
The RMSE of df_6_0_2: 0.6381305250841128
The RMSE of df_6_0_3: 0.5590922790681273
The RMSE of df_6_0_4: 0.5701509630096697
The RMSE of df_6_0_5: 0.7391717017239601
The RMSE of df_6_0_6: 0.8625822530745677
The RMSE of df_6_0_7: 0.9454847576446683
The RMSE of df_6_0_8: 0.987105825236917
The RMSE of df_6_0_9: 1.164295983705952
The RMSE of df_6_1_1: 1.780001188556762
The RMSE of df_6_1_2: 1.8279238400339766
The RMSE of df_6_1_3: 1.7330520261946645
The RMSE of df_6_1_4: 1.7217037002654718
The RMSE of df_6_1_5: 1.7901137769272275
The RMSE of df_6_1_6: 1.7719790773655753
The RMSE of df_6_1_7: 1.706660019941872
The RMSE of df_6_1_8: 1.7556458406979705
The RMSE of df_6_1_9: 1.852179185171505


In [ ]:
df_6_features_3_1 = df_6_3_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_2 = df_6_3_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_3 = df_6_3_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_4 = df_6_3_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_5 = df_6_3_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_6 = df_6_3_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_7 = df_6_3_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_8 = df_6_3_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_9 = df_6_3_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_3_10 = df_6_3_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
user_similarity(df_6_features_3_1, df_6_3_1)
user_similarity(df_6_features_3_2, df_6_3_2)
user_similarity(df_6_features_3_3, df_6_3_3)
user_similarity(df_6_features_3_4, df_6_3_4)
user_similarity(df_6_features_3_5, df_6_3_5)
user_similarity(df_6_features_3_6, df_6_3_6)
user_similarity(df_6_features_3_7, df_6_3_7)
user_similarity(df_6_features_3_8, df_6_3_8)
user_similarity(df_6_features_3_9, df_6_3_9)
user_similarity(df_6_features_3_10, df_6_3_10)

The RMSE of df_6_3_1: nan
The RMSE of df_6_3_2: nan
The RMSE of df_6_3_3: nan
The RMSE of df_6_3_4: nan
The RMSE of df_6_3_5: nan
The RMSE of df_6_3_6: nan
The RMSE of df_6_3_7: nan
The RMSE of df_6_3_8: nan
The RMSE of df_6_3_9: nan


In [ ]:
df_6_features_5_1 = df_6_5_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_2 = df_6_5_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_3 = df_6_5_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_4 = df_6_5_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_5 = df_6_5_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_6 = df_6_5_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_7 = df_6_5_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_8 = df_6_5_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_9 = df_6_5_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_10 = df_6_5_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_11 = df_6_5_11.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_12 = df_6_5_12.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_13 = df_6_5_13.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_14 = df_6_5_14.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_15 = df_6_5_15.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_16 = df_6_5_16.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_17 = df_6_5_17.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_18 = df_6_5_18.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_19 = df_6_5_19.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
df_6_features_5_20 = df_6_5_20.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0)
user_similarity(df_6_features_5_1, df_6_5_1)
user_similarity(df_6_features_5_2, df_6_5_2)
user_similarity(df_6_features_5_3, df_6_5_3)
user_similarity(df_6_features_5_4, df_6_5_4)
user_similarity(df_6_features_5_5, df_6_5_5)
user_similarity(df_6_features_5_6, df_6_5_6)

The RMSE of df_6_5_1: 1.0253953216713527
The RMSE of df_6_5_2: 1.171059046552048
The RMSE of df_6_5_3: 0.990590720158057
The RMSE of df_6_5_4: 0.8767936170046646
The RMSE of df_6_5_5: 0.9075142884209343
The RMSE of df_6_5_6: 0.9785100995528015


In [ ]:
user_similarity(df_6_features_5_7, df_6_5_7)
user_similarity(df_6_features_5_8, df_6_5_8)
user_similarity(df_6_features_5_9, df_6_5_9)
user_similarity(df_6_features_5_10, df_6_5_10)
user_similarity(df_6_features_5_11, df_6_5_11)
user_similarity(df_6_features_5_12, df_6_5_12)
user_similarity(df_6_features_5_13, df_6_5_13)
user_similarity(df_6_features_5_14, df_6_5_14)
user_similarity(df_6_features_5_15, df_6_5_15)
user_similarity(df_6_features_5_16, df_6_5_16)
user_similarity(df_6_features_5_17, df_6_5_17)
user_similarity(df_6_features_5_18, df_6_5_18)
user_similarity(df_6_features_5_19, df_6_5_19)
user_similarity(df_6_features_5_20, df_6_5_20)

The RMSE of df_6_5_7: 0.9808209780282106
The RMSE of df_6_5_8: 1.2313007251683648
The RMSE of df_6_5_9: 1.0264175915975924
The RMSE of df_6_5_10: 1.2579953553228302
The RMSE of df_6_5_11: 1.3101304123344413
The RMSE of df_6_5_12: 1.061560874571706
The RMSE of df_6_5_13: 1.0618885529908844
The RMSE of df_6_5_14: 1.1623833501381564
The RMSE of df_6_5_15: 1.1686063830996491
The RMSE of df_6_5_16: 1.2594810440142328
The RMSE of df_6_5_17: 1.3205440921886387
The RMSE of df_6_5_18: 1.2184841245943847
The RMSE of df_6_5_19: 1.0519840173645698
The RMSE of df_6_5_20: 0.6655959641737935


In [ ]:
user_similarity(df_6_features_2, df_6_2)
user_similarity(df_6_features_4, df_6_4)

The RMSE of df_6_2: 0.0
The RMSE of df_6_4: 0.0


### Item Similarity

In [ ]:
def item_similarity(df_features, original_df):
  mean = np.nanmean(df_features, axis=1)
  df_subtracted = (df_features.T-mean).T
# Item Similarity Matrix
  item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
  item_correlation[np.isnan(item_correlation)] = 0
  item_correlation[item_correlation<0]=0
  item_predicted_ratings = np.dot((df_features.fillna(0).T),item_correlation)
  dummy_df = original_df.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)
  item_final_rating = np.multiply(item_predicted_ratings,dummy_df)
  X  = item_final_rating.copy() 
  X = X[X>0]
  scaler = MinMaxScaler(feature_range=(1, 5))
  scaler.fit(X)
  y = (scaler.transform(X))
  df_ = original_df.pivot_table(index='User-ID',columns='ISBN',values='Book-Rating')
  # Finding total non-NaN value

  total_non_nan = np.count_nonzero(~np.isnan(y))
  rmse = (sum(sum((df_ - y )**2))/total_non_nan)**0.5
  print("{} (Item) RMSE: {}".format(get_df_name(original_df), rmse))

In [ ]:
df_6_features_2 = df_6_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_4 = df_6_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
item_similarity(df_6_features_2, df_6_2)
item_similarity(df_6_features_4, df_6_4)

df_6_2 (Item) RMSE: 0.0
df_6_4 (Item) RMSE: 1.1091400327377785


In [ ]:
df_6_features_0_1 = df_6_0_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_2 = df_6_0_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_3 = df_6_0_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_4 = df_6_0_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_5 = df_6_0_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_6 = df_6_0_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_7 = df_6_0_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_8 = df_6_0_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_9 = df_6_0_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_0_10 = df_6_0_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
item_similarity(df_6_features_0_1, df_6_0_1)
item_similarity(df_6_features_0_2, df_6_0_2)
item_similarity(df_6_features_0_3, df_6_0_3)
item_similarity(df_6_features_0_4, df_6_0_4)
item_similarity(df_6_features_0_5, df_6_0_5)
item_similarity(df_6_features_0_6, df_6_0_6)
item_similarity(df_6_features_0_7, df_6_0_7)
item_similarity(df_6_features_0_8, df_6_0_8)
item_similarity(df_6_features_0_9, df_6_0_9)

df_6_0_1 (Item) RMSE: 1.7168510070148157
df_6_0_2 (Item) RMSE: 1.6178417426038179
df_6_0_3 (Item) RMSE: 1.6649895086712134
df_6_0_4 (Item) RMSE: 1.6089827813983493
df_6_0_5 (Item) RMSE: 1.6405199179002383
df_6_0_6 (Item) RMSE: 1.6756292491376388
df_6_0_7 (Item) RMSE: 1.611184110286575
df_6_0_8 (Item) RMSE: 1.583975288383375
df_6_0_9 (Item) RMSE: 1.5205257246992552


In [ ]:
df_6_features_1_1 = df_6_1_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_2 = df_6_1_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_3 = df_6_1_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_4 = df_6_1_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_5 = df_6_1_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_6 = df_6_1_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_7 = df_6_1_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_8 = df_6_1_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_9 = df_6_1_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_1_10 = df_6_0_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
item_similarity(df_6_features_1_1, df_6_1_1)
item_similarity(df_6_features_1_2, df_6_1_2)
item_similarity(df_6_features_1_3, df_6_1_3)
item_similarity(df_6_features_1_4, df_6_1_4)
item_similarity(df_6_features_1_5, df_6_1_5)
item_similarity(df_6_features_1_6, df_6_1_6)
item_similarity(df_6_features_1_7, df_6_1_7)
item_similarity(df_6_features_1_8, df_6_1_8)
item_similarity(df_6_features_1_9, df_6_1_9)

df_6_1_1 (Item) RMSE: 1.7402631806401403
df_6_1_2 (Item) RMSE: 1.6925632327521622
df_6_1_3 (Item) RMSE: 1.7154715451990572
df_6_1_4 (Item) RMSE: 1.71147995040021
df_6_1_5 (Item) RMSE: 1.749439850243652
df_6_1_6 (Item) RMSE: 1.709693595648483
df_6_1_7 (Item) RMSE: 1.6659059278050812
df_6_1_8 (Item) RMSE: 1.6435216863807518
df_6_1_9 (Item) RMSE: 1.590461742314615


In [ ]:
df_6_features_3_1 = df_6_3_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_2 = df_6_3_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_3 = df_6_3_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_4 = df_6_3_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_5 = df_6_3_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_6 = df_6_3_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_7 = df_6_3_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_8 = df_6_3_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_9 = df_6_3_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_10 = df_6_3_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_11 = df_6_3_11.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_12 = df_6_3_12.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_13 = df_6_3_13.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_14 = df_6_3_14.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_3_15 = df_6_3_15.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
item_similarity(df_6_features_3_1, df_6_3_1)
item_similarity(df_6_features_3_2, df_6_3_2)
item_similarity(df_6_features_3_3, df_6_3_3)
item_similarity(df_6_features_3_4, df_6_3_4)
item_similarity(df_6_features_3_5, df_6_3_5)
item_similarity(df_6_features_3_6, df_6_3_6)
item_similarity(df_6_features_3_7, df_6_3_7)
item_similarity(df_6_features_3_8, df_6_3_8)
item_similarity(df_6_features_3_9, df_6_3_9)
item_similarity(df_6_features_3_10, df_6_3_10)
item_similarity(df_6_features_3_11, df_6_3_11)
item_similarity(df_6_features_3_12, df_6_3_12)
item_similarity(df_6_features_3_13, df_6_3_13)
item_similarity(df_6_features_3_14, df_6_3_14)
item_similarity(df_6_features_3_15, df_6_3_15)

df_6_3_1 (Item) RMSE: nan
df_6_3_2 (Item) RMSE: nan
df_6_3_3 (Item) RMSE: nan
df_6_3_4 (Item) RMSE: nan
df_6_3_5 (Item) RMSE: nan
df_6_3_6 (Item) RMSE: nan
df_6_3_7 (Item) RMSE: nan
df_6_3_8 (Item) RMSE: nan
df_6_3_9 (Item) RMSE: nan
df_6_3_10 (Item) RMSE: nan
df_6_3_11 (Item) RMSE: nan
df_6_3_12 (Item) RMSE: nan
df_6_3_13 (Item) RMSE: nan
df_6_3_14 (Item) RMSE: nan
df_6_3_15 (Item) RMSE: nan


In [ ]:
df_6_features_5_1 = df_6_5_1.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_2 = df_6_5_2.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_3 = df_6_5_3.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_4 = df_6_5_4.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_5 = df_6_5_5.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_6 = df_6_5_6.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_7 = df_6_5_7.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_8 = df_6_5_8.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_9 = df_6_5_9.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_10 = df_6_5_10.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_11 = df_6_5_11.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_12 = df_6_5_12.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_13 = df_6_5_13.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_14 = df_6_5_14.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_15 = df_6_5_15.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_16 = df_6_5_16.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_17 = df_6_5_17.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_18 = df_6_5_18.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_19 = df_6_5_19.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
df_6_features_5_20 = df_6_5_20.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating',fill_value=0).T
item_similarity(df_6_features_5_1, df_6_5_1)
item_similarity(df_6_features_5_2, df_6_5_2)
item_similarity(df_6_features_5_3, df_6_5_3)
item_similarity(df_6_features_5_4, df_6_5_4)
item_similarity(df_6_features_5_5, df_6_5_5)
item_similarity(df_6_features_5_6, df_6_5_6)

df_6_5_1 (Item) RMSE: 1.1361718144629633
df_6_5_2 (Item) RMSE: 1.1608390966964266
df_6_5_3 (Item) RMSE: 1.1128200469956286
df_6_5_4 (Item) RMSE: 0.9156631000185861
df_6_5_5 (Item) RMSE: 1.0495781833995825
df_6_5_6 (Item) RMSE: 0.9865853246243731


In [ ]:
item_similarity(df_6_features_5_7, df_6_5_7)
item_similarity(df_6_features_5_8, df_6_5_8)
item_similarity(df_6_features_5_9, df_6_5_9)
item_similarity(df_6_features_5_10, df_6_5_10)
item_similarity(df_6_features_5_11, df_6_5_11)
item_similarity(df_6_features_5_12, df_6_5_12)
item_similarity(df_6_features_5_13, df_6_5_13)
item_similarity(df_6_features_5_14, df_6_5_14)
item_similarity(df_6_features_5_15, df_6_5_15)
item_similarity(df_6_features_5_16, df_6_5_16)
item_similarity(df_6_features_5_17, df_6_5_17)
item_similarity(df_6_features_5_18, df_6_5_18)
item_similarity(df_6_features_5_19, df_6_5_19)
item_similarity(df_6_features_5_20, df_6_5_20)

df_6_5_7 (Item) RMSE: 1.0084824925937488
df_6_5_8 (Item) RMSE: 1.2439963238163387
df_6_5_9 (Item) RMSE: 1.0430918233348352
df_6_5_10 (Item) RMSE: 1.2881065350273582
df_6_5_11 (Item) RMSE: 1.324102142895921
df_6_5_12 (Item) RMSE: 1.0625155986513524
df_6_5_13 (Item) RMSE: 1.0620162791542131
df_6_5_14 (Item) RMSE: 1.1752153235042928
df_6_5_15 (Item) RMSE: 1.1104534715214307
df_6_5_16 (Item) RMSE: 1.1322928548384052
df_6_5_17 (Item) RMSE: 1.156240804211554
df_6_5_18 (Item) RMSE: 1.1119502032620352
df_6_5_19 (Item) RMSE: 0.990552038746308
df_6_5_20 (Item) RMSE: 0.6368584784792508


### SVD + Cross Validation

In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_6_0[["User-ID", "ISBN", "Book-Rating"]], reader)

param_grid = {
    "n_epochs": [1, 5, 10, 15, 20],
    "lr_all": [0.002, 0.005, 0.009, 0.01, 0.05],
    "reg_all": [0.2, 0.4, 0.6, 1]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

0.22453306365472472


In [ ]:
data = Dataset.load_from_df(df_6_1[["User-ID", "ISBN", "Book-Rating"]], reader)

param_grid = {
    "n_epochs": [1, 5, 10, 15, 20],
    "lr_all": [0.002, 0.005, 0.009, 0.01, 0.05],
    "reg_all": [0.2, 0.4, 0.6, 1]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

0.22517708359085487


In [ ]:
data = Dataset.load_from_df(df_6_4[["User-ID", "ISBN", "Book-Rating"]], reader)

param_grid = {
    "n_epochs": [1, 5, 10, 15, 20],
    "lr_all": [0.002, 0.005, 0.009, 0.01, 0.05],
    "reg_all": [0.2, 0.4, 0.6, 1]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

0.6621372986908082


In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_6_2[["User-ID", "ISBN", "Book-Rating"]], reader)

param_grid = {
    "n_epochs": [1, 5, 10, 15, 20],
    "lr_all": [0.002, 0.005, 0.009, 0.01, 0.05],
    "reg_all": [0.2, 0.4, 0.6, 1]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

0.582062940212878


In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_6_3[["User-ID", "ISBN", "Book-Rating"]], reader)

param_grid = {
    "n_epochs": [1, 5, 10, 15, 20],
    "lr_all": [0.002, 0.005, 0.009, 0.01, 0.05],
    "reg_all": [0.2, 0.4, 0.6, 1]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

1.0


In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_6_5[["User-ID", "ISBN", "Book-Rating"]], reader)

param_grid = {
    "n_epochs": [1, 5, 10, 15, 20],
    "lr_all": [0.002, 0.005, 0.009, 0.01, 0.05],
    "reg_all": [0.2, 0.4, 0.6, 1]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

0.6351974715878365
